In [1]:
#!pip install fake_useragent

In [1]:
import pandas as pd
import numpy as np
import requests
import time
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from tqdm import tqdm_notebook

In [2]:
# подгрузим один из методов этой библиотеки
from fake_useragent import UserAgent

In [3]:
UserAgent().chrome

'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'

## Описание задачи

Парсинг сайта https://topliba.com/ с целью сбора данных (рейтинг, название фильма, описание, жанр, год выпуска) по жанру Фэнтези. Дело в том, что большинство современных книжный сайтов не предоставляют возможности делать сквозной  поиск по всем страницам, а только в рамках делений по жанрам. Наиболее представленный жанр н данном сайте - фэнтези (783 страницы поиска) по нему и будем делать парсинг. На данном сайте список фильмов подгружается динамически по страницам. Обратимся к странице Фэнтези, и запишем запрос к данной странице.

In [4]:
page_link = 'https://topliba.com/genres/12090?p=1'

In [5]:
response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})
response

<Response [200]>

In [6]:
html = response.content

### Получение ссылок с фильмами

С помощью библиотеки BeautifulSoup распарсим страницу и найдем ссылки книгам. 

In [7]:
soup = BeautifulSoup(response.content, 'html.parser')

In [8]:
response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})
html = response.content
html[:1000]

b'<!DOCTYPE html>\n<html lang="ru">\n    <head>\n        <meta charset="utf-8">\n        <meta http-equiv="X-UA-Compatible" content="IE=edge">\n        <meta name="viewport" content="width=device-width, initial-scale=1">\n        <title>\xd0\xa4\xd1\x8d\xd0\xbd\xd1\x82\xd0\xb5\xd0\xb7\xd0\xb8: \xd1\x81\xd0\xba\xd0\xb0\xd1\x87\xd0\xb0\xd1\x82\xd1\x8c \xd0\xba\xd0\xbd\xd0\xb8\xd0\xb3\xd0\xb8 \xd0\xb2 fb2, \xd1\x87\xd0\xb8\xd1\x82\xd0\xb0\xd1\x82\xd1\x8c \xd0\xbe\xd0\xbd\xd0\xbb\xd0\xb0\xd0\xb9\xd0\xbd \xe2\x80\xa2 \xd0\xa1\xd1\x82\xd1\x80\xd0\xb0\xd0\xbd\xd0\xb8\xd1\x86\xd0\xb0 1</title>\n        <meta name="description" content="\xd0\x9a\xd0\xbd\xd0\xb8\xd0\xb3\xd0\xb8 \xd0\xb6\xd0\xb0\xd0\xbd\xd1\x80\xd0\xb0 \xd0\xa4\xd1\x8d\xd0\xbd\xd1\x82\xd0\xb5\xd0\xb7\xd0\xb8 \xd0\xb4\xd0\xbe\xd1\x81\xd1\x82\xd1\x83\xd0\xbf\xd0\xbd\xd1\x8b\xd0\xb5 \xd0\xb2 \xd1\x84\xd0\xbe\xd1\x80\xd0\xbc\xd0\xb0\xd1\x82\xd0\xb5 fb2, \xd0\xb0 \xd1\x82\xd0\xb0\xd0\xba\xd0\xb6\xd0\xb5 \xd0\xb4\xd0\xbb\xd1\x8f \xd1\x

In [9]:
book_container = soup.findAll(lambda tag: tag.name == 'a' and tag.get('class') == ['book-title'])
book_container[:6]

[<a class="book-title" href="https://topliba.com/books/898377" title="Скачать книгу Новая жизнь темного властелина. Часть 1">Новая жизнь темного властелина. Часть 1</a>,
 <a class="book-title" href="https://topliba.com/books/898358" title="Скачать книгу Не выпускайте чудовищ из шкафа">Не выпускайте чудовищ из шкафа</a>,
 <a class="book-title" href="https://topliba.com/books/898208" title="Скачать книгу Земная">Земная</a>,
 <a class="book-title" href="https://topliba.com/books/898197" title="Скачать книгу Проклятая">Проклятая</a>,
 <a class="book-title" href="https://topliba.com/books/898032" title="Скачать книгу Этот лот (не) продается!!!">Этот лот (не) продается!!!</a>,
 <a class="book-title" href="https://topliba.com/books/897776" title="Скачать книгу Дитя солнца">Дитя солнца</a>]

In [10]:
# Cписок ссылок на каждую книгу со страницы
book_links = [link.attrs['href'] for link in 
                  soup.findAll(lambda tag: tag.name == 'a' and tag.get('class') == ['book-title'])]
book_links[:6]

['https://topliba.com/books/898377',
 'https://topliba.com/books/898358',
 'https://topliba.com/books/898208',
 'https://topliba.com/books/898197',
 'https://topliba.com/books/898032',
 'https://topliba.com/books/897776']

На сайте https://topliba.com/ book_links уже выводятся полностью, вместе с названием сайта, поэтому вместо ['https://topliba.com' + link.attrs['href'] for link in book_links] будем подставлять только ['link.attrs['href'] for link in book_links]

In [11]:
def getPageLinks(page_number): # функция для вытаскивания ссылок на конкретные книги,
                               # возвращает список из ссылок на книги, полученные с текущей страницы  
                               # Параметры: page_number: int/string - номер страницы для парсинга            
    
    # составляем ссылку на страницу поиска
    page_link = 'https://topliba.com/genres/12090?p={}'.format(page_number)
    
    # запрашиваем данные о ней
    response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})
        
    if not response.ok:
        # если сервер нам отказал, вернем пустой лист для текущей страницы
        print('Сервер ничего не вернул')
        return [] 
    
    # получаем содержимое страницы и переводим в суп
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # ищем ссылки на книги и формируем полные ссылки
    book_links = soup.findAll(lambda tag: tag.name == 'a' and tag.get('class') == ['book-title'])
    book_links = [link.attrs['href'] for link in book_links]
    
    return book_links

In [12]:
# book_links = getPageLinks(30)
# book_links

### Получим информации о книге
Обратимся к странице с фильмом Сердце Дракона. Том 2 и достанем следующую информацию:
- Название книги.
- Автор книги
- Оценка книги
- Аннотация/Описание
- Количество читателей
- Количество комментариев/отзывов

In [13]:
book_page = 'https://topliba.com/books/736796'
response = requests.get(book_page, headers={'User-Agent': UserAgent().chrome})

html = response.content
soup = BeautifulSoup(response.content, 'html.parser')

### Получим название книги

In [14]:
book_title = soup.find(lambda tag: tag.name == 'div' and tag.get('class') == ['row'])
book_title

<div class="row">
<div class="header-background"><div style="background: url(https://topliba.com/covers/736796.jpg?t=1705317373)"></div></div>
<div class="col-xs-12 col-sm-3 cover">
<div itemscope="" itemtype="http://schema.org/ImageObject" style="position: relative">
<meta content="Лисина Александра - Щит императора" itemprop="name">
<img alt="Лисина Александра - Щит императора" itemprop="contentUrl" src="https://topliba.com/covers/736796.jpg?t=1705317373" title="Лисина Александра - Щит императора"/>
</meta></div>
</div>
<div class="col-xs-12 col-sm-9 book-title">
<h1 itemprop="name">Щит императора</h1>
<h2 class="book-author" itemprop="author">
<a href="https://topliba.com/authors/2142" title="Книги Александра Лисина">Александра Лисина</a>
</h2>
</div>
</div>

Заголовок книги находится в теге h1, который имеет атрибут itemprop="name". 

In [15]:
book_title = soup.find('h1', itemprop='name').text
book_title

'Щит императора'

In [16]:
#book_title = soup.find(lambda tag: tag.name == 'h1') # and tag.get('itemprop') == ['name'])
#book_title = film_title.text
#book_title

### Получим автора книги

In [17]:
book_author = soup.find(lambda tag: tag.name == 'h2' and tag.get('class') == ['book-author'])
book_author = book_author.text.replace("\n", "")
book_author

'Александра Лисина'

### Получим аннотацию к книге

In [18]:
book_announs = soup.find(lambda tag: tag.name == 'div' and tag.get('class') == ['description']).text
book_announs = book_announs.strip()
book_announs

'После смерти великого императора Орриана на престол вступает его единственный сын, которому в наследство достается не только страна, но и телохранитель. Карриану не нужен такой довесок к короне, однако нарушить волю отца он не рискует. Мару тоже не нравится это решение, но приказ хозяина для него превыше собственных предпочтений. Для двух врагов подобное соседство — худшее, что только можно измыслить. Тень, не желающая оберегать нового императора, и император, ненавидящий свою тень… Казалось бы, у этих двоих нет и не может быть ничего общего. Но жизнь все расставляет по своим местам.'

### Получим рейтинг книги 

Рейтинги книг хранятся в аттрибутах тега div class="col-lg-12". Посмотрим на структуру данного тега.

In [19]:
scoreboard = soup.find(lambda tag: tag.name == 'div' and tag.get('class') == ['col-lg-12'])
scoreboard

<div class="col-lg-12">
<rating _avgrating="4.75" _rating="0" _ratingsnumber="251" _showavgrating="true" _showavgratingtext="true" _url="/books/736796/ratings" _user="" class="pull-left"></rating>
<meta content="1" itemprop="worstRating"/>
<meta content="5" itemprop="bestRating"/>
<meta content="251" itemprop="ratingCount"/>
<meta content="4.75" itemprop="ratingValue"/>
</div>

In [20]:
def getRating(soup, rate):
    try:
        obj = soup.find('rating', attrs={'class':'pull-left'})
        obj = obj.get(rate)
    except:
        obj=None
    
    return obj

In [21]:
getRating(soup, '_avgrating')

'4.75'

### Получим количество читателей книги и количества комментариев (отзывов)

Сведения о читателях хранятся в book-readers

In [22]:
book_readers = soup.find(lambda tag: tag.name == 'div' and tag.get('class') == ['book-readers'])
book_readers

<div class="book-readers">
<div class="col-lg-8 col-sm-12 well-sm">
<div class="col-sm-2 col-xs-4 book-readers-status">
<span class="book-readers-status-count">490</span>
<span>читателей</span>
</div>
<div class="col-sm-2 col-xs-4 book-readers-status">
<a href="https://topliba.com/books/736796/impressions">
<span class="book-readers-status-count">88</span>
<span>отзывов</span>
</a>
</div>
<div class="col-sm-2 col-xs-4 book-readers-status">
<a href="https://topliba.com/books/736796/quotes">
<span class="book-readers-status-count">20</span>
<span>цитат</span>
</a>
</div>
</div>
</div>

In [23]:
# Находим все span элементы с классом 'book-readers-status-count'
readers_count_elements = soup.find_all('span', class_='book-readers-status-count')

book_readership_count = readers_count_elements[0].string  
book_comment_count = readers_count_elements[1].string  

print("Количество читателей:", book_readership_count)
print("Количество отзывов:", book_comment_count)

Количество читателей: 490
Количество отзывов: 88


### Получение данных о жанре и годе издания

In [24]:
# Находим все теги div с классом 'col-lg-12'
# search_elements = soup.find_all('div', class_='col-lg-12')
# search_elements

In [25]:
"""
genre = search_elements[1].text.replace('\n', '')  
year = search_elements[2].text.replace('\n', '')

print("Жанры:", genre)
print("Год:", year)
"""

'\ngenre = search_elements[1].text.replace(\'\n\', \'\')  \nyear = search_elements[2].text.replace(\'\n\', \'\')\n\nprint("Жанры:", genre)\nprint("Год:", year)\n'

## Запишем итоговую функцию для сбора данных по книге
ссылку вида https://topliba.com/books/893256 передаем в getBookData(book_page) и получаем **карточку данной книги**

In [26]:
def getBookData(book_page):
    book_data = pd.DataFrame(columns=['Title', 'Author', 'Description', 'Raiting', 'Readers', 'Comments'], index=[0])
    
    response = requests.get(book_page, headers={'User-Agent': UserAgent().chrome})
    
    if not response.ok:
        # если сервер нам отказал, вернем статус ошибки 
        return response.status_code
    
    # получаем содержимое страницы и переводим в суп
    soup = BeautifulSoup(response.content, 'html.parser')
     
    book_title = soup.find('h1', itemprop='name').text
    book_data['Title'] = book_title  
                
    book_author = soup.find(lambda tag: tag.name == 'h2' and tag.get('class') == ['book-author'])
    book_author = book_author.text.replace("\n", "")
    book_data['Author'] = book_author
    
    book_announs = soup.find(lambda tag: tag.name == 'div' and tag.get('class') == ['description']).text
    book_announs = book_announs.replace('\n', '')
    book_announs = book_announs.strip()
    book_data['Description']= book_announs
      
    book_data['Raiting'] = getRating(soup=soup, rate='_avgrating')
    
    readers_count_elements = soup.find_all('span', class_='book-readers-status-count')

    book_readership_count = readers_count_elements[0].string  
    book_data['Readers'] = book_readership_count
    
    book_comment_count = readers_count_elements[1].string  
    book_data['Comments'] = book_comment_count
    
    #search_elements = soup.find_all('div', class_='col-lg-12')
    #genre = search_elements[1].text
    #book_data['Genre']=genre
    
    return book_data

In [27]:
getBookData('https://topliba.com/books/736796')

,Title,Author,Description,Raiting,Readers,Comments
0,Щит императора,Александра Лисина,После смерти великого императора Орриана на пр...,4.75,490,88


## Получение информации по всем книгам

Соберем информацию по всем книгам. Cоздадим пустой датафрейм, в который мы будем добавлять информацию по каждой книге.

In [28]:
final_df = pd.DataFrame(columns=['Title', 'Author', 'Description', 'Raiting', 'Readers', 'Comments'])
final_df.head()

,Title,Author,Description,Raiting,Readers,Comments


### Страница https://topliba.com/genres/12090?p=784 дает ссылки на 784 страницах, на каждой странице по 24 ссылки на книгу. Итого будет 18 816 ссылок на книги

In [29]:
# Цикл А.Ч.
def getAllBooks(num=1000):
    book_data_list = []
    for i in range(num):
        links = getPageLinks(num)
        for j in links:
            book_data = getBookData(j)
            book_data_list.extend(book_data)
            time.sleep(0.5)
    final_df.to_csv('books_{}.csv'.format(page_number), index=False) # сохранение CSV без индексов
    return final_df
final_df.to_csv('books_full.csv')

In [30]:
data = pd.read_csv('books_full.csv', index_col=0)
data.head(60)

,Title,Author,Description,Raiting,Readers,Comments


# Второй способ создания цикла - по лекции Марии Тихоновой.

In [31]:
def getBookData(book_page):
    # запрашиваем данные по ссылке
    response = requests.get(book_page, headers={'User-Agent': UserAgent().chrome})

    if not response.ok:
        # если сервер нам отказал, вернем статус ошибки
        return response.status_code

    # получаем содержимое страницы и переводим в суп
    html = response.content
    soup = BeautifulSoup(html,'html.parser')

    # используя ранее написанные функции парсим информацию
    book_title = soup.find('h1', itemprop='name').text
                 
    book_author = soup.find(lambda tag: tag.name == 'h2' and tag.get('class') == ['book-author'])
    book_author = book_author.text.replace("\n", "")
    
    book_announs = soup.find(lambda tag: tag.name == 'div' and tag.get('class') == ['description']).text
    book_announs = book_announs.replace('\n', '')
    book_announs = book_announs.strip()
         
    raiting = getRating(soup=soup, rate='_avgrating')
    
    readers_count_elements = soup.find_all('span', class_='book-readers-status-count')

    book_readership_count = readers_count_elements[0].string  
        
    book_comment_count = readers_count_elements[1].string  
           
    # составляем словарь, в котором будут хранится все полученные и обработанные данные
    data_row = {'Title':book_title, 'Author':book_author,
                'Description':book_announs, 'Raiting':raiting, 'Readers':book_readership_count,'Comments':book_comment_count}
    return data_row

In [32]:
data_row = getBookData('https://topliba.com/books/736796')

In [33]:
data_row

{'Title': 'Щит императора',
 'Author': 'Александра Лисина',
 'Description': 'После смерти великого императора Орриана на престол вступает его единственный сын, которому в наследство достается не только страна, но и телохранитель. Карриану не нужен такой довесок к короне, однако нарушить волю отца он не рискует. Мару тоже не нравится это решение, но приказ хозяина для него превыше собственных предпочтений. Для двух врагов подобное соседство — худшее, что только можно измыслить. Тень, не желающая оберегать нового императора, и император, ненавидящий свою тень… Казалось бы, у этих двоих нет и не может быть ничего общего. Но жизнь все расставляет по своим местам.',
 'Raiting': '4.75',
 'Readers': '490',
 'Comments': '88'}

In [34]:
final_df = pd.DataFrame(columns=['Title', 'Author', 'Description', 'Raiting', 'Readers', 'Comments'])

In [35]:
final_df

,Title,Author,Description,Raiting,Readers,Comments


In [36]:
book_links

['https://topliba.com/books/898377',
 'https://topliba.com/books/898358',
 'https://topliba.com/books/898208',
 'https://topliba.com/books/898197',
 'https://topliba.com/books/898032',
 'https://topliba.com/books/897776',
 'https://topliba.com/books/898031',
 'https://topliba.com/books/897772',
 'https://topliba.com/books/897639',
 'https://topliba.com/books/897636',
 'https://topliba.com/books/897633',
 'https://topliba.com/books/897397',
 'https://topliba.com/books/897225',
 'https://topliba.com/books/897388',
 'https://topliba.com/books/897224',
 'https://topliba.com/books/897276',
 'https://topliba.com/books/897272',
 'https://topliba.com/books/897246',
 'https://topliba.com/books/897229',
 'https://topliba.com/books/897209',
 'https://topliba.com/books/896911',
 'https://topliba.com/books/896731',
 'https://topliba.com/books/897016',
 'https://topliba.com/books/896710']

In [37]:
from tqdm.notebook import tqdm
for book_link in tqdm(book_links):
    try:
        data_row = getBookData(book_link)
        final_df = final_df.append(data_row, ignore_index=True)
        time.sleep(0.3)
    except:
        continue

  0%|          | 0/24 [00:00<?, ?it/s]

C:\Users\Olga\AppData\Local\Temp\ipykernel_1200\133204077.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(data_row, ignore_index=True)
C:\Users\Olga\AppData\Local\Temp\ipykernel_1200\133204077.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(data_row, ignore_index=True)
C:\Users\Olga\AppData\Local\Temp\ipykernel_1200\133204077.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(data_row, ignore_index=True)
C:\Users\Olga\AppData\Local\Temp\ipykernel_1200\133204077.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(data_row,

In [38]:
final_df.head(60)

,Title,Author,Description,Raiting,Readers,Comments
0,Новая жизнь темного властелина. Часть 1,Андрей Федин,В своих прошлых жизнях я много внимания уделял...,4.00,9,0
1,Не выпускайте чудовищ из шкафа,Карина Дёмина,"Остров Дальний - место, где некогда добывали м...",3.16,62,3
2,Земная,Jude Moon,"С самого детства я знала, что мне уготована уч...",3.67,13,0
3,Проклятая,Jude Moon,"Я мечтала попасть в другой мир, но не думала, ...",3.50,34,1
4,Этот лот (не) продается!!!,Ника Лисицына,Стоило только подписать свой тысячный контракт...,3.38,50,0
5,Дитя солнца,Рони,"Каждому агенту, рано или поздно, предстоит пер...",5.00,1,0
6,Проклятье чёрной луны,Ника Лисицына,"Я – ведьма, способная видеть будущее. Вот толь...",3.92,74,1
7,Проклятие гостеприимства,Pauli Bal,"Если в родной дом пришла беда, даже самые край...",3.50,5,0
8,За лунным цветком,Мряу Пушистая,"Иногда добыть что-то несравнимо легче, чем дон...",4.00,4,0
9,"Бедность, Война и Любовь",Ленор Клемм,"Говорят, что для полноты своей жизни человек д...",2.67,4,0


In [39]:
# Цикл А.Ч.
def getAllBooks(num=1000):
    book_data_list = []
    for i in range(num):
        links = getPageLinks(num)
        for j in links:
            book_data = getBookData(j)
            book_data_list.extend(book_data)
            time.sleep(0.5)
    final_df.to_csv('books_{}.csv'.format(page_number), index=False) # сохранение CSV без индексов
    return final_df
final_df.to_csv('books_full.csv')

In [40]:
data = pd.read_csv('books_full.csv', index_col=0)
data.head(60)

,Title,Author,Description,Raiting,Readers,Comments
0,Новая жизнь темного властелина. Часть 1,Андрей Федин,В своих прошлых жизнях я много внимания уделял...,4.00,9,0
1,Не выпускайте чудовищ из шкафа,Карина Дёмина,"Остров Дальний - место, где некогда добывали м...",3.16,62,3
2,Земная,Jude Moon,"С самого детства я знала, что мне уготована уч...",3.67,13,0
3,Проклятая,Jude Moon,"Я мечтала попасть в другой мир, но не думала, ...",3.50,34,1
4,Этот лот (не) продается!!!,Ника Лисицына,Стоило только подписать свой тысячный контракт...,3.38,50,0
5,Дитя солнца,Рони,"Каждому агенту, рано или поздно, предстоит пер...",5.00,1,0
6,Проклятье чёрной луны,Ника Лисицына,"Я – ведьма, способная видеть будущее. Вот толь...",3.92,74,1
7,Проклятие гостеприимства,Pauli Bal,"Если в родной дом пришла беда, даже самые край...",3.50,5,0
8,За лунным цветком,Мряу Пушистая,"Иногда добыть что-то несравнимо легче, чем дон...",4.00,4,0
9,"Бедность, Война и Любовь",Ленор Клемм,"Говорят, что для полноты своей жизни человек д...",2.67,4,0
